In [ ]:
from dotenv import load_dotenv
import os

llm_api_key = os.getenv("ANTHROPIC_API_KEY")
api_max_tokens = 750


path_to_initPrompt_1 = "Prompts\initial_prompt_1.txt"
path_to_pdf = "ReqView-Example_Software_Requirements_Specification_SRS_Document.pdf"
path_to_initPrompt_2 = "Prompts\initial_prompt_2.txt"
path_to_reflectionPrompt = "Prompts\reflection_prompt.txt"
path_to_finalPrompt = "Prompts\final_prompt.txt"

path_to_output = "output_test_matrix.json"

In [4]:
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(
    model="claude-3-7-sonnet-latest",
    temperature = 0.3,
    max_tokens = api_max_tokens,
    api_key = llm_api_key
)

In [5]:
from typing import TypedDict, Annotated
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]


In [ ]:
## content_prompt_initial
import base64

with open(path_to_pdf, "rb") as f:
    pdf_data = base64.standard_b64encode(f.read()).decode("utf-8")

with open(path_to_initPrompt_1,"r") as f:
    initPrompt_1 = f.read()

with open(path_to_initPrompt_2,"r") as f:
    initPrompt_2 = f.read()

content_prompt_initial = [
                {
                    "type": "text",
                    "text": initPrompt_1
                },
                {
                    "type": "document",
                    "source": {
                        "type": "base64",
                        "media_type": "application/pdf",
                        "data": pdf_data
                    },
                    "cache_control": {"type": "ephemeral"}
                },
                {
                    "type": "text",
                    "text": initPrompt_2
                }
            ]

In [ ]:
## content_prompt_reflection
with open(path_to_reflectionPrompt,"r") as f:
    reflectionPrompt = f.read()

content_prompt_reflection = [
                {
                    "type": "text",
                    "text": reflectionPrompt
                }
            ]

In [ ]:
## content_prompt_final
with open(path_to_finalPrompt,"r") as f:
    finalPrompt = f.read()
content_prompt_final = [
                {
                    "type": "text",
                    "text": finalPrompt
                }
            ]

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage

def write_initial_draft(state: AgentState):
    model_response = llm.invoke([HumanMessage(content=content_prompt_initial)])
    
    return {"messages": [HumanMessage(content=content_prompt_initial), model_response]}
    

In [ ]:
def write_reflection(state: AgentState):
    prompt_messages = state['messages'] + [HumanMessage(content=content_prompt_reflection)] 
    model_response = llm.invoke(prompt_messages)

    return{"messages": [HumanMessage(content=content_prompt_reflection), model_response]}

In [ ]:
def write_final(state: AgentState):
    prompt_messages = state['messages'] + [HumanMessage(content=content_prompt_final)] +  [AIMessage(content="{")]
    ## The AIMessage { is to tell Claude to only create JSON
    model_response = llm.invoke(prompt_messages)

    return{"messages":[HumanMessage(content=content_prompt_final), model_response]}

In [15]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(AgentState)
builder.add_node(write_initial_draft, "write_initial_draft")
builder.add_node(write_reflection, "write_reflection")
builder.add_node(write_final, "write_final")

builder.add_edge(START, "write_initial_draft")
builder.add_edge("write_initial_draft","write_reflection")
builder.add_edge("write_reflection","write_final")
builder.add_edge("write_final",END)


graph = builder.compile()

In [ ]:
langchainConfig_messages = graph.invoke({"messages": []})

for message in langchainConfig_messages['messages']:
    print(message)

APIConnectionError: Connection error.

In [ ]:
import json

output_testCase_JSON = langchainConfig_messages['messages'][0]
with open(path_to_output, "w") as file:
    json.dump(output_testCase_JSON, file, indent=4, ensure_ascii=False)